In [6]:
import numpy as np
import pymongo

In [7]:
min_services = 2
max_services = 10 

min_acceptable_latency = 1
max_acceptable_latency = 20

min_avg_instructions_per_second = 100000
max_avg_instructions_per_second = 1000000

# Define the number of configurations (n)
n = 100

In [8]:
client_url = "mongodb://localhost:27017/"
db_name = "simulations"
collection_name = "services"

In [9]:
# Initialize a dictionary to store the generated service configurations
service_configurations = []

# Connect to MongoDB
client = pymongo.MongoClient(client_url)
db = client[db_name]
collection = db[collection_name] 

# Generate service configurations for each network configuration
for i in range(n):
    config_id = f'config-{i + 1}'  # Unique configuration ID for the network configuration
    
    # Generate a random number of services (m) within the defined range
    num_services = np.random.randint(min_services, max_services + 1)
    
    service_configs = []
    for j in range(num_services):
        service_id = f'service-{j + 1}'  # Unique service ID
        
        # Generate attributes for the service
        acceptable_latency = np.random.uniform(min_acceptable_latency, max_acceptable_latency)
        avg_instructions_per_second = np.random.uniform(min_avg_instructions_per_second, max_avg_instructions_per_second)
        
        # Create the service configuration as a dictionary
        service_config = {
            '_id': service_id,
            'acceptable_latency': acceptable_latency,
            'average_instructions_per_second': avg_instructions_per_second
        }
        
        service_configs.append(service_config)
    
    # Create the service configuration for the network configuration
    network_service_config = {
        '_id': config_id,
        'services': service_configs
    }
    
    # Insert the service configuration into MongoDB
    collection.insert_one(network_service_config)
    service_configurations.append(network_service_config)

# Close the MongoDB connection
client.close()

In [12]:
class Service:
    def __init__(self, data):
        self.id = data['_id']
        self.acceptable_latency = data['acceptable_latency']
        self.average_instructions_per_second = data['average_instructions_per_second']

# Define a class to represent a Service Configuration
class ServiceConfiguration:
    def __init__(self, data):
        self.id = data['_id']
        self.services = [Service(service_data) for service_data in data['services']]

In [13]:
# Define a class to manage Service Configurations
class ServiceConfigurationManager:
    def __init__(self, client_url, db_name, collection_name):
        self.client = pymongo.MongoClient(client_url)  # Change the connection string as needed
        self.db = self.client[db_name]
        self.collection = self.db[collection_name]

    def get_configuration_by_id(self, config_id):
        config_data = self.collection.find_one({'_id': config_id})
        if config_data:
            return ServiceConfiguration(config_data)
        else:
            return None

    def get_all_configurations(self):
        configurations = []
        cursor = self.collection.find({})
        for config_data in cursor:
            configurations.append(ServiceConfiguration(config_data))
        return configurations

    def close_connection(self):
        self.client.close()

In [14]:
service_config_manager = ServiceConfigurationManager(client_url, db_name, collection_name)

In [15]:
all_service_configs = service_config_manager.get_all_configurations()
for i, config in enumerate(all_service_configs, start=1):
    print(f"Service Configuration {i}:")
    for service in config.services:
        print(f"Service {service.id}:")
        print(f"Acceptable Latency: {service.acceptable_latency}")
        print(f"Average Instructions per Second: {service.average_instructions_per_second}")

Service Configuration 1:
Service service-1:
Acceptable Latency: 7.689172716932043
Average Instructions per Second: 150847.09952115858
Service service-2:
Acceptable Latency: 16.9146368410643
Average Instructions per Second: 514951.45806609537
Service service-3:
Acceptable Latency: 8.421624114691566
Average Instructions per Second: 560783.4316872925
Service service-4:
Acceptable Latency: 14.350465571735253
Average Instructions per Second: 369937.38913598476
Service service-5:
Acceptable Latency: 3.1580129431073267
Average Instructions per Second: 149448.78668422776
Service service-6:
Acceptable Latency: 14.177276674033033
Average Instructions per Second: 199780.95090395008
Service service-7:
Acceptable Latency: 3.777943552161815
Average Instructions per Second: 335669.258674803
Service Configuration 2:
Service service-1:
Acceptable Latency: 10.77491859984298
Average Instructions per Second: 102195.50150263471
Service service-2:
Acceptable Latency: 4.413921885733493
Average Instructions p